# Intervalos

**[1]** (i) Define un tipo compósito de Julia Intervalo para representar un intervalo de dos números reales, que incluya redondeo dirigido. Por el momento supongamos que todos los elementos serán de tipo BigFloat.

Para ello, deberás definir constructores que acepten cadenas y/o números.

(ii) Implementa las operaciones básicas sobre intervalos.

(iii) Limpia tu código para reducir la repetición de código, al sacar cada vez código en común a una función por separado.

**R:** Para la primera parte este es el codigo que nos crea el intervalo ya considerando el redondeo.

In [2]:
type Intervalo
    inf::BigFloat
    sup::BigFloat

    function Intervalo(x,y)
        #En esta parte simplemente nos aseguramos que las entradas sean intervalos
        if x>=y
            error("Las entradas no corresponden a un intervalo")
        end
        #Esta es la parte fina del codigo donde vamos a redondear
        xs=string(x)
        ys=string(y)
        set_rounding(Float64,RoundDown)
        x=BigFloat(float(xs))
        set_rounding(Float64,RoundUp)
        y=BigFloat(float(ys))
        new(x,y)
    end
end

In [17]:
#Razonamiento para crear la funcion
x=0.1
BigFloat(x)
y=string(x)
set_rounding(Float64,RoundDown)
println(BigFloat(float(y)))
set_rounding(Float64,RoundUp)
println(BigFloat(float(y)))

9.999999999999999167332731531132594682276248931884765625e-02
1.000000000000000055511151231257827021181583404541015625e-01


In [5]:
A=Intervalo(0.1,0.3)#Funciona el redondeo

Intervalo(9.999999999999999167332731531132594682276248931884765625e-02 with 256 bits of precision,3.000000000000000444089209850062616169452667236328125e-01 with 256 bits of precision)

Ahora vamos a definir las operaciones con intervalos:

In [3]:
#Suma
function +(A::Intervalo,B::Intervalo)
    C=Intervalo(0.0,0.1)
    C.inf=A.inf+B.inf
    C.sup=A.sup+B.sup
    return C
end

+ (generic function with 216 methods)

In [4]:
#Resta
function -(A::Intervalo,B::Intervalo)
    C=Intervalo(0.0,0.1)
    C.inf=A.inf-B.sup
    C.sup=A.sup-B.inf
    return C
end

- (generic function with 242 methods)

In [6]:
#Producto
function *(A::Intervalo,B::Intervalo)
    C=Intervalo(0.0,0.1)
    #Primer caso: A positivo, B positivo
    if (A.inf > 0.0) & (A.sup > 0.0) & (B.inf > 0.0) & (B.sup > 0.0)    
        C.inf=A.sup*B.sup
        C.sup=A.inf*B.inf
    #Segundo caso: A positivo, B negativo
    elseif (A.inf > 0.0) & (A.sup > 0.0) & (B.inf < 0.0) & (B.sup < 0.0)
        C.inf=A.sup*B.inf
        C.sup=A.inf*B.sup
    #Tercer caso: A positivo, B negativo y positivo
    elseif (A.inf > 0.0) & (A.sup > 0.0) & (B.inf < 0.0) & (B.sup > 0.0)
        C.inf=A.sup*B.inf
        C.sup=A.sup*B.sup
    #Cuarto caso: A negativo, B negativo
    elseif (A.inf < 0.0) & (A.sup < 0.0) & (B.inf < 0.0) & (B.sup < 0.0)
        C.inf=A.sup*B.sup
        C.sup=A.inf*B.inf
    #Quinto caso: A negativo y positivo, B negativo y positivo
    elseif (A.inf < 0.0) & (A.sup > 0.0) & (B.inf < 0.0) & (B.sup > 0.0)
        if abs(A.sup)>=abs(A.inf)
            C.inf=A.sup*B.inf
            C.sup=A.sup*B.sup
        else
            C.inf=A.inf*B.sup
            C.sup=A.inf*B.inf
        end
    #Sexto caso: A negativo, B positivo
    elseif (A.inf < 0.0) & (A.sup < 0.0) & (B.inf > 0.0) & (B.sup > 0.0)
        C.inf=A.inf*B.sup
        C.sup=A.sup*B.inf
    #Septimo caso: A negativo y positivo, B positivo
    elseif (A.inf < 0.0) & (A.sup > 0.0) & (B.inf > 0.0) & (B.sup > 0.0)
        C.inf=A.inf*B.sup
        C.sup=A.sup*B.sup
    #Ultimo caso: A negativo y positivo, B negativo
    else
        C.inf=A.sup*B.inf
        C.sup=A.inf*B.inf
    end
    return C
end

* (generic function with 278 methods)

La funcion anterior falla en una de las condiciones, se propone la siguiente solucion alternativa "ligeramente" mas corta:

In [5]:
function *(A::Intervalo,B::Intervalo)
    Valores=[A.inf*B.inf A.inf*B.sup A.sup*B.inf A.sup*B.sup]
    C=Intervalo(minimum(Valores),maximum(Valores))
    return C
end

* (generic function with 278 methods)

In [21]:
A=Intervalo(-1,0.1)
B=Intervalo(-0.1,2)

Intervalo(-9.999999999999999167332731531132594682276248931884765625e-02 with 256 bits of precision,2e+00 with 256 bits of precision)

In [27]:
A*B#Aqui funciona incluso el caso que destruia mi codigo anterior

Intervalo(-2e+00 with 256 bits of precision,2.00000000000000011102230246251565404236316680908203125e-01 with 256 bits of precision)

In [6]:
function /(A::Intervalo,B::Intervalo)
    if (B.inf < 0.0) & (B.sup > 0.0)#podria ahorrarme una operacion, Como?
        println("Tu intervalo del denominador contiene al 0, no se puede calcular el intervalo")
    else
    Valores=[A.inf/B.inf A.inf/B.sup A.sup/B.inf A.sup/B.sup]
    C=Intervalo(minimum(Valores),maximum(Valores))
    end
    return C
end

/ (generic function with 73 methods)

In [41]:
D=Intervalo(-9,-1)
E=Intervalo(-3,0)
D/E

Intervalo(-inf with 256 bits of precision,3e+00 with 256 bits of precision)

**[2]** Haz un módulo de Julia llamado Intervalos en un archivo intervalos.jl, que contiene todas las definiciones anteriores.

**R:** El modulo fue creado con el nombre intervalos.jl

**[3]** (i) Escribe tests ("pruebas") usando FactCheck.jl.

Estos tests se deberán correr cada vez que modifiques tu código, ¡para verificar que no lo hayas estropeado entre tanto!

(ii) ¡Intenta destruir el código de alguien más! O sea, escribe tests que realmente prueben el código del otro (¡"extreme testing"!)

In [47]:
Pkg.add("FactCheck")

INFO: Nothing to be done


In [50]:
using FactCheck
facts("Esto es una prueba") do
    1>=1
    2*2>=4
end


Esto es una prueba
0 facts verified.


delayed_handler (generic function with 4 methods)

# Operaciones con intervalos



**[5]** 

(i) Define la potencia para intervalos.

(ii) Para el intervalo X=[−1,1], calcula X⋅X y $X^2$ (donde ⋅ denota la multiplicación de intervalos). ¿Qué observas?

(iii) Calcula [−1,1]⋅([−1,0]+[3,4]) y [−1,1]⋅[−1,0]+[−1,1]⋅[3,4]. ¿Qué observas?


**R:** Definiremos la potencia a partir de la multiplicación que ya programé. 

In [7]:
function ^(A::Intervalo)
    C=A
    for i=1:n
        C=C*A#Utilizamos la operacion multiplicación programada previamente
    end
    return C
end

^ (generic function with 54 methods)

In [8]:
A^2

Intervalo(9.99999999999999673871986516360266250558197498321533203125e-03 with 256 bits of precision,9.000000000000003830269434956790064461529254913330078125e-02 with 256 bits of precision)

In [9]:
A*A#Comprobamos que hace lo que queremos

Intervalo(9.99999999999999673871986516360266250558197498321533203125e-03 with 256 bits of precision,9.000000000000003830269434956790064461529254913330078125e-02 with 256 bits of precision)

Ahora Realizamos las operaciones con el intervalo X dado $( X=[-1,1] )$

In [9]:
X=Intervalo(-1,1)

Intervalo(-1e+00 with 256 bits of precision,1e+00 with 256 bits of precision)

In [11]:
X*X

Intervalo(-1e+00 with 256 bits of precision,1e+00 with 256 bits of precision)

In [12]:
X^2

Intervalo(-1e+00 with 256 bits of precision,1e+00 with 256 bits of precision)

Los resultados son los mismos y esto era de esperarse, pues para programar la función potencia se utilizó la función producto programad apreviamente.

Finalmente para los cálculos que nos solicitan:

In [21]:
Q=Intervalo(-1,0)
R=Intervalo(3,4)
X*(Q+R)

Intervalo(-4e+00 with 256 bits of precision,4e+00 with 256 bits of precision)

In [22]:
X*Q+X*R

Intervalo(-5e+00 with 256 bits of precision,5e+00 with 256 bits of precision)

No se satisface la distributividad, es decir $A(B+C)$ es distinto de $AB+AC$

**[6]** Con las operaciones definidas en Julia, podemos insertar intervalos en cálculos sencillos.

(i) Define una función polinomial $p_1(x):=(x−1)(x−2)$.

(ii) ¿Cómo se puede mandar a $p_1$ un intervalo como argumento?

(iii) ¿Qué representa el resultado?

(iv) Juega con distintos intervalos como entrada y dibuja los resultados. ¿Qué observas?

(v) ¿Qué pasa si reescribes $p_1$ en una forma equivalente? ¿En otra forma equivalente?

(vi) Pensando en este ejemplo de polinomios, ¿para qué nos sirven los intervalos?

**R:** Primero definimos la función polinomial conocida:

In [18]:
function poli(x)
    p=(x-1)*(x-2)
    return p
end

poli (generic function with 1 method)

In [33]:
poli(3)

2

Intentamos probar un Intervalo en nuestra función:

In [35]:
poli(X)

LoadError: `-` has no method matching -(::Intervalo, ::Int64)
while loading In[35], in expression starting on line 1

Proponemos una nueva función polinomial que tiene como argumento un intervalo y devuelve otro intervalo:

In [19]:
function poli2(X::Intervalo)
    A=Intervalo(0,0.1)
    B=Intervalo(0,0.1)
    A.inf=X.inf-1
    A.sup=X.sup-1#(x-1)
    B.inf=X.inf-2
    B.sup=X.sup-2#(x-2)
    P=A*B#p=(x-1)(x-2)
    return P
end

poli2 (generic function with 1 method)

In [21]:
poli2(X)

Intervalo(-0e+00 with 256 bits of precision,6e+00 with 256 bits of precision)

Lo que obtenemos por resultado representa el polinomio evaluado en los extremos del intervalo, **¿es esto correcto?**

Ahora propondremos escribir $p_1(x)=(x-1)(x-2)=x^2-3x+2$, que es una forma equivalente. Una segunda forma sería $p_1(x)=x(x-3)+2$. Vamos a evaluar los resultados:

In [24]:
function poli3(X::Intervalo)
    P=X^2-X-X-X
    P.inf=P.inf+2
    P.sup=P.sup+2
    return P
end

poli3 (generic function with 1 method)

In [25]:
poli3(X)

Intervalo(-2e+00 with 256 bits of precision,6e+00 with 256 bits of precision)

In [26]:
function poli4(X::Intervalo)
    D=Intervalo(0,0.1)
    D.inf=D.inf-3
    D.sup=D.sup-3
    P=X*D
    P.inf=P.inf+2
    P.sup=P.sup+2
    return P
end

poli4 (generic function with 1 method)

In [27]:
poli4(X)

Intervalo(-1e+00 with 256 bits of precision,5e+00 with 256 bits of precision)

Con cada forma que escribimos el polinomio obtenemos distintos resultados, esto por la distributividad.

¿Para que podríamos usarlo? Es una manera de manipular los límites de un intervalo con el que estemos trabajando.

**[7]**

(i) Haz una implementación de tu idea de la pregunta [6].

(ii) Pruébalo con $p_2(x):=x^2−2$.

(iii) Pruébalo con otros polinomios.

In [8]:
function poli5(X::Intervalo)#X^2-2
    P=X^2
    P.inf=P.inf-2
    P.sup=P.sup-2
    return P
end

poli5 (generic function with 1 method)

In [10]:
poli5(X)

Intervalo(-3e+00 with 256 bits of precision,-1e+00 with 256 bits of precision)